In [1]:
import numpy as np
#chain and combinations needed for powerset
from itertools import chain, combinations
from sympy import poly
from sympy.abc import x,y
from sympy import symbols, Poly
##last model

#defining my class of graphs that can have loops or parallel edges
class MultiGraph:
    """A class for undirected graphs"""
    def __init__(self,V=set(), E=dict()):
        #self.incident- dictionary with edges incident to the specific vertex (key)
        self.incident=dict((v,set()) for v in V)
        for e in E:
            u,w=E[e]
            self.incident[u].add(e)
            self.incident[w].add(e)
        #self.edges - dictionary with vertices that form the edge (key)
        self.edges=E
    def VertexSet(self):
        #returns the vertex set
        return set(self.incident)
    def EdgeSet(self):
        #returns the edge set
        return set(self.edges)
    def Order(self):
        return len(self.VertexSet())
    def Size(self):
        return len(self.EdgeSet())
    def Neighbours(self,v):
        #neighbours of a vertex
        b=[]
        #goes through all the edges and checks if at least one end vertex is the same as the one requested
        for e in self.edges:
            u,w=self.edges[e]
            if v==u:
                b.append(w)
            elif v==w:
                b.append(u)
        return b
    def adjiacent(self,u,v):
        #checks if two vertices are adjacent
        return u in self.Neighbours(v)

    def count_components(self,edges):
        #counts number of components given the edges
        #visited is a set with all the visited (gone through the code) vertices
        visited = set()
        components = 0
        for v in self.VertexSet():
            if v not in visited:
                #if the vertex is not visited, then the number of components goes up by one
                components += 1
                #performing the depth first search for the vertex
                self.dfs(v, visited,edges)
        return components

    def dfs(self, v, visited,edges):
        #depth first search
        #add the vertex to the visited ones
        visited.add(v)
        #checking if the vertex has any incident vertex
        for e in self.incident[v]:
            #u and w are the vertices that make the edge
            u, w = self.edges[e]
            #checking if we have the edge in the edge set given in the definition call, if not, continue/break
            if e not in edges:
                continue
            # if the vertex that we started from (given in the dfs) is one of the end vertex of the edge we are on
            if v == u:
                #then if the other end of the edge has not yet been visited, perform the dfs on it
                if w not in visited:
                    self.dfs(w, visited,edges)
            #the other way around, same procedure
            else:
                if u not in visited:
                    self.dfs(u, visited,edges)

    def powerset(self,iterable):
        s=iterable
        #making a powerset out of a set
        #need to make sure iterable is a list (chat gbt says it can be a set too)
        return list(chain.from_iterable(combinations(s,r) for r in range(len(s)+1)))

    def add_edges(self, edges):
        #adds edges to the graph, edge should be given as a dictionary
        for e, (u, w) in edges.items():
            self.incident[u].add(e)
            self.incident[w].add(e)
            self.edges[e] = (u, w)

    def remove_edges(self, edges):
        #removes edges, edge should be given as a list [edges]
        #removes the edges from the incident dictionary and then removes them from the edges dictionary
        for e in edges:
            if e in self.edges:
                u, w = self.edges[e]
                if u==w:
                    self.incident[u].remove(e)
                else:
                    self.incident[u].remove(e)
                    self.incident[w].remove(e)
                del self.edges[e]

    def contract_edges(self,edge):
        #contracting one edge
        if edge not in self.edges:
            print("The specified edge does not exist in the graph.")
            return
        #u and w are the vertices that form the edge that is contracted
        u,w=self.edges[edge]
        #if we have a loop, remove the edge only from that vertex
        if u==w:
            self.incident[u].remove(edge)
        #if we have a normal edge, remove it from both vertices incident dictionary
        else:
            self.incident[u].remove(edge)
            self.incident[w].remove(edge)
        #remove the edge from the edges dictionary
        del self.edges[edge]
        #update the incident dictionary for one of the vertices by adding the edges from the second one
        self.incident[u].update(self.incident[w])
        del self.incident[w]
        #going through the edges dictionary and making changes so that one of the vertices is removed
        for e in self.edges:
            a,b=self.edges[e]
            #put it as a set because tuples cannot be changed
            x=list(self.edges[e])
            if a==w:
                x.remove(a)
                x.append(u)
            if b==w:
                x.remove(b)
                x.append(u)
            #rewrite the new edges
            self.edges[e]=tuple(x)

        #print("Edge contracted successfully.")

    def edge_type(self,edge):
        #what type of edge is the specific edge
        u,w=self.edges[edge]
        #if the end vertices are the same, it is a loop
        if u==w:
            t='loop'
            return t
        #if the number of components changes when the edge removed, it is a bridge
        graph=self.deleted_graph(edge)
        n=graph.count_components(graph.EdgeSet())
        if self.count_components(self.EdgeSet())!=n:
            t='bridge'
            return t
        #otherwise it is a loop
        t='ordinary'
        return t

    def nr_ordinary(self):
        #counts number of ordinary edges
        a=0
        for e in self.EdgeSet():
            if self.edge_type(e)=='ordinary':
                a+=1
        return a

    def nr_loop(self):
        #counts number of loops
        a=0
        for e in self.EdgeSet():
            if self.edge_type(e)=='loop':
                a+=1
        return a

    def nr_bridge(self):
        #counts number of bridges
        a=0
        for e in self.EdgeSet():
            if self.edge_type(e)=='bridge':
                a+=1
        return a

    def first_ordinary_edge(self):
        #returns the first ordinary edge in the edge list
        if self.nr_ordinary()>=1:
            for i in range(self.Size()-1):
                edge = list(self.EdgeSet())[i]
                if self.edge_type(edge)=='ordinary':
                    return edge

    def copy(self):
        # Create a copy of the graph
        V = self.VertexSet()
        E = self.edges
        return MultiGraph(V,E)

    def contracted_graph(self, edge):
        # Create a copy of the graph and contracts the specified edge
        contracted_graph = MultiGraph(set(self.VertexSet()), dict(self.edges))
        contracted_graph.contract_edges(edge)
        return contracted_graph

    def deleted_graph(self, edge):
        # Create a copy of the graph and removes the specified edge
        deleted_graph = MultiGraph(set(self.VertexSet()), dict(self.edges))
        deleted_graph.remove_edges([edge])
        return deleted_graph

    def graph_with_subset_edges(self,edgeset):
        #creates a new graph with a subset of edges from the initial graph
        edge_dict={edge:self.edges[edge] for edge in edgeset}
        graph=MultiGraph(self.VertexSet(),edge_dict)
        return graph

    def nr_forests(self): #for t(2,1)
        #calculates the number of forests in a graph (i.e. acyclic edge subsets)
        pset=self.powerset(self.EdgeSet())
        result = 0
        for subset in pset:
            graph=self.graph_with_subset_edges(subset)
            if graph.has_cycle()==False:
                result+=1
        return result

    def nr_spanning_forests(self): #for t(1,1)
        #calculates the number of spanning forests (acyclic graph with subset edges that have the same nr of components as the initial graph
        pset=self.powerset(self.EdgeSet())
        result=0
        for subset in pset:
            graph=self.graph_with_subset_edges(subset)
            n_g=graph.count_components(graph.EdgeSet())
            if graph.has_cycle()==False:
                if self.count_components(self.EdgeSet())==n_g:
                    result+=1
        return result

    def nr_spanning_subgraphs(self): #for t(1,2)
        #calculates the number of spanning subgraphs (subgraphs that have the same number of components)
        pset=self.powerset(self.EdgeSet())
        result=0
        for subset in pset:
            graph=self.graph_with_subset_edges(subset)
            n_g=graph.count_components(graph.EdgeSet())
            if self.count_components(self.EdgeSet())==n_g:
                result+=1
        return result


    def has_cycle(self):
        #checks is a graph has a cycle of at least 3 vertices
        visited = set()
        for v in self.VertexSet():
            if v not in visited:
                if self.dfs_cycle(v, visited, parent=None, path_length=0):
                    return True
        return False

    def dfs_cycle(self, v, visited, parent, path_length):
        #depth first search for a cycle of at least length 3
        visited.add(v)
        for neighbor in self.Neighbours(v):
            if neighbor not in visited:
                if self.dfs_cycle(neighbor, visited, v, path_length + 1):
                    return True
            elif neighbor != parent and path_length >= 2:
                return True
        return False

    def tutte_polynomial_by_contract_remove_edges(self,x,y):
        x_sym, y_sym = symbols('x y')
        #calculates the tutte polynomial by the edge contracting/deleting recurring formula
        if self.nr_ordinary()==0:
            return x ** self.nr_bridge() * y ** self.nr_loop()
        if self.Size()==0:
            return 1
        edge=self.first_ordinary_edge()
        contracted_graph = self.contracted_graph(edge)
        deleted_graph = self.deleted_graph(edge)
        pol=deleted_graph.tutte_polynomial_by_contract_remove_edges(x, y) + contracted_graph.tutte_polynomial_by_contract_remove_edges( x, y)
        polynomial = Poly(pol, x_sym, y_sym)
        polynomial = polynomial.subs(x_sym, x).subs(y_sym, y)
        return polynomial


    def tutte_polynomial_by_number_of_components(self, x, y):
        pset = self.powerset(self.edges.keys())
        result = 0
        k_e = self.count_components(self.edges.keys())  # Number of components of the whole graph

        # Define the symbolic variables
        x_sym, y_sym = symbols('x y')

        for subset in pset:
            k_a = self.count_components(subset)  # Number of components with the given edge set
            result += (x_sym - 1)**(k_a - k_e) * (y_sym - 1)**(k_a + len(subset) - self.Order())

        # Create a polynomial from the result using the symbolic variables
        polynomial = Poly(result, x_sym, y_sym)
        polynomial = polynomial.subs(x_sym, x).subs(y_sym, y)

        return polynomial


Types of graphs: polynomials given in literature and its calculations with my program

1. Empty/null graph

In [6]:
import pandas as pd

In [8]:
E1=MultiGraph(('A'),{})
E5=MultiGraph(('A','B','C','D','E'),{})
E10=MultiGraph(('A','B','C','D','E','F','G','H','I','J'),{})
P2=MultiGraph(('A','B'),{1:('A','B')})
P5=MultiGraph(('A','B','C','D','E'),{1:('A','B'),2:('B','C'),3:('C','D'),4:('D','E')})
P10=MultiGraph(('A','B','C','D','E','F','G','H','I','J'),{1:('A','B'),2:('B','C'),3:('C','D'),4:('D','E'),5:('E','F'),6:('F','G'),7:('G','H'),8:('H','I'),9:('I','J')})
C10=MultiGraph(('A','B','C','D','E','F','G','H','I','J'),{1:('A','B'),2:('B','C'),3:('C','D'),4:('D','E'),5:('E','F'),6:('F','G'),7:('G','H'),8:('H','I'),9:('I','J'),10:('J','A')})
P8=MultiGraph(('A','B','C','D','E','F','G','H'),{1:('A','B'),2:('B','C'),3:('C','D'),4:('D','E'),5:('E','F'),6:('F','G'),7:('G','A'),8:(('A','H'))})
Sl10=MultiGraph(('A','B','C','D','E','F','G','H','I','J'),{1:('A','B'),2:('B','C'),3:('C','D'),4:('D','E'),5:('E','A'),6:('A','F'),7:('B','G'),8:('C','H'),9:('D','I'),10:('E','J')})
St7=MultiGraph(('A','B','C','D','E','F','G','H'),{1:('A','B'),2:('A','C'),3:('A','D'),4:('A','E'),5:('A','F'),6:('A','G'),7:('A','H')})
Bull_graph=MultiGraph(('A','B','C','D','E'),{1:('A','B'),2:('B','C'),3:('C','A'),4:('A','D'),5:('B','E')})
Butterfly_graph=MultiGraph(('A','B','C','D','E'),{1:('A','B'),2:('A','C'),3:('B','C'),4:('C','D'),5:('C','E'),6:('D','E')})
Cubical_graph=MultiGraph(('A','B','C','D','E','F','G','H'),{1:('A','B'),2:('C','D'),3:('E','F'),4:('G','H'),5:('A','G'),6:('B','H'),7:('F','G'),8:('E','H'),9:('C','F'),10:('D','E'),11:('A','C'),12:('B','D')})
Diamond_graph=MultiGraph(('A','B','C','D'),{1:('A','B'),2:('B','C'),3:('C','D'),4:('D','A'),5:('B','D')})
L5=MultiGraph(('A','B','C','D','E','X','Y','Z','W','V'),{1:('A','B'),2:('B','C'),3:('C','D'),4:('D','E'),5:('X','Y'),6:('Y','Z'),7:('Z','W'),8:('W','V'),9:('A','X'),10:('B','Y'),11:('C','Z'),12:('D','W'),13:('E','V')})
W6=MultiGraph(('A','B','C','D','E','F'),{1:('A','B'),2:('B','C'),3:('C','D'),4:('D','E'),5:('E','A'),6:('A','F'),7:('B','F'),8:('C','F'),9:('D','F'),10:('E','F')})
G4=MultiGraph(('A','B','C','D','E','F','G','H','I'),{1:('A','B'),2:('B','C'),3:('C','D'),4:('D','E'),5:('E','F'),6:('F','G'),7:('G','H'),8:('H','A'),9:('A','I'),10:('C','I'),11:('E','I'),12:('G','I')})

In [9]:
Graphs=[E1,E5,E10,P2,P5,P10,C10,P8,Sl10,St7,Bull_graph,Butterfly_graph,Cubical_graph,Diamond_graph,L5,W6,G4]

In [20]:
a={'Graphs':['Empty graph (n=1)','Empty graph (n=5)','Empty graph (n=10)','Path graph P2','Path graph P5','Path graph P10','Cycle graph C10','Pan graph C7+K1','Sunlet graph (n=10)','Star graph S7','Bull graph','Butterfly graph','Cubical graph','Diamond graph','Ladder graph (n=5)','Wheel graph (n=6)','Gear graph (n=4)'],'Tutte Polynomial':[],'T(1,1)':[],'T(1,2)':[],'T(2,1)':[],'T(2,2)':[]}

In [21]:
for i in range(len(Graphs)):
    G=Graphs[i]
    if G.tutte_polynomial_by_number_of_components(x,y)==G.tutte_polynomial_by_contract_remove_edges(x,y):
        a['Tutte Polynomial'].append(G.tutte_polynomial_by_number_of_components(x,y))
    if G.tutte_polynomial_by_number_of_components(x,y)(1,1)==G.nr_spanning_forests():
        a['T(1,1)'].append(G.tutte_polynomial_by_number_of_components(1,1))
    if G.tutte_polynomial_by_number_of_components(x,y)(1,2)==G.nr_spanning_subgraphs():
        a['T(1,2)'].append(G.tutte_polynomial_by_number_of_components(1,2))
    if G.tutte_polynomial_by_number_of_components(x,y)(2,1)==G.nr_forests():
        a['T(2,1)'].append(G.tutte_polynomial_by_number_of_components(2,1))
    if G.tutte_polynomial_by_number_of_components(x,y)(2,2)==2**(G.Size()):
        a['T(2,2)'].append(G.tutte_polynomial_by_number_of_components(2,2))
a

{'Graphs': ['Empty graph (n=1)',
  'Empty graph (n=5)',
  'Empty graph (n=10)',
  'Path graph P2',
  'Path graph P5',
  'Path graph P10',
  'Cycle graph C10',
  'Pan graph C7+K1',
  'Sunlet graph (n=10)',
  'Star graph S7',
  'Bull graph',
  'Butterfly graph',
  'Cubical graph',
  'Diamond graph',
  'Ladder graph (n=5)',
  'Wheel graph (n=6)',
  'Gear graph (n=4)'],
 'Tutte Polynomial': [Poly(1, x, y, domain='ZZ'),
  Poly(1, x, y, domain='ZZ'),
  Poly(1, x, y, domain='ZZ'),
  Poly(x, x, y, domain='ZZ'),
  Poly(x**4, x, y, domain='ZZ'),
  Poly(x**9, x, y, domain='ZZ'),
  Poly(x**9 + x**8 + x**7 + x**6 + x**5 + x**4 + x**3 + x**2 + x + y, x, y, domain='ZZ'),
  Poly(x**7 + x**6 + x**5 + x**4 + x**3 + x**2 + x*y, x, y, domain='ZZ'),
  Poly(x**9 + x**8 + x**7 + x**6 + x**5*y, x, y, domain='ZZ'),
  Poly(x**7, x, y, domain='ZZ'),
  Poly(x**4 + x**3 + x**2*y, x, y, domain='ZZ'),
  Poly(x**4 + 2*x**3 + 2*x**2*y + x**2 + 2*x*y + y**2, x, y, domain='ZZ'),
  Poly(x**7 + 5*x**6 + 15*x**5 + 6*x**4*y

In [11]:
for i in range(len(Graphs)):
    G=Graphs[i]
    if G.tutte_polynomial_by_number_of_components(x,y)(1,1)==G.nr_spanning_forests():
        a['T(1,1)'].append(G.tutte_polynomial_by_number_of_components(1,1))
a

{'Graphs': ['Empty graph (n=1)',
  'Empty graph (n=5)',
  'Empty graph (n=10)',
  'Path graph P2',
  'Path graph P5',
  'Path graph P10',
  'Cycle graph C10',
  'Pan graph C7+K1',
  'Sunlet graph (n=10)',
  'Star graph S7',
  'Bull graph',
  'Butterfly graph',
  'Cubical graph',
  'Diamond graph',
  'Ladder graph (n=5)',
  'Wheel graph (n=6)',
  'Gear graph (n=4)'],
 'T(1,1)': [1, 1, 1, 1, 1, 1, 10, 7, 5, 1, 3, 9, 384, 8, 209, 121, 192],
 'T(1,2)': [],
 'T(2,1)': [],
 'T(2,2)': []}

In [12]:
for i in range(len(Graphs)):
    G=Graphs[i]
    if G.tutte_polynomial_by_number_of_components(x,y)==G.tutte_polynomial_by_contract_remove_edges(x,y) and G.tutte_polynomial_by_number_of_components(x,y)(1,2)==G.nr_spanning_subgraphs():
        a['T(1,2)'].append(G.tutte_polynomial_by_number_of_components(1,2))
a

{'Graphs': ['Empty graph (n=1)',
  'Empty graph (n=5)',
  'Empty graph (n=10)',
  'Path graph P2',
  'Path graph P5',
  'Path graph P10',
  'Cycle graph C10',
  'Pan graph C7+K1',
  'Sunlet graph (n=10)',
  'Star graph S7',
  'Bull graph',
  'Butterfly graph',
  'Cubical graph',
  'Diamond graph',
  'Ladder graph (n=5)',
  'Wheel graph (n=6)',
  'Gear graph (n=4)'],
 'T(1,1)': [1, 1, 1, 1, 1, 1, 10, 7, 5, 1, 3, 9, 384, 8, 209, 121, 192],
 'T(1,2)': [1, 1, 1, 1, 1, 1, 11, 8, 6, 1, 4, 16, 1083, 14, 479, 462, 431],
 'T(2,1)': [],
 'T(2,2)': []}

In [13]:
for i in range(len(Graphs)):
    G=Graphs[i]
    if G.tutte_polynomial_by_number_of_components(x,y)==G.tutte_polynomial_by_contract_remove_edges(x,y) and G.tutte_polynomial_by_number_of_components(x,y)(2,1)==G.nr_forests():
        a['T(2,1)'].append(G.tutte_polynomial_by_number_of_components(2,1))
a

{'Graphs': ['Empty graph (n=1)',
  'Empty graph (n=5)',
  'Empty graph (n=10)',
  'Path graph P2',
  'Path graph P5',
  'Path graph P10',
  'Cycle graph C10',
  'Pan graph C7+K1',
  'Sunlet graph (n=10)',
  'Star graph S7',
  'Bull graph',
  'Butterfly graph',
  'Cubical graph',
  'Diamond graph',
  'Ladder graph (n=5)',
  'Wheel graph (n=6)',
  'Gear graph (n=4)'],
 'T(1,1)': [1, 1, 1, 1, 1, 1, 10, 7, 5, 1, 3, 9, 384, 8, 209, 121, 192],
 'T(1,2)': [1, 1, 1, 1, 1, 1, 11, 8, 6, 1, 4, 16, 1083, 14, 479, 462, 431],
 'T(2,1)': [1,
  1,
  1,
  2,
  16,
  512,
  1023,
  254,
  992,
  128,
  28,
  49,
  2656,
  24,
  6240,
  462,
  3102],
 'T(2,2)': []}

In [14]:
for i in range(len(Graphs)):
    G=Graphs[i]
    if G.tutte_polynomial_by_number_of_components(x,y)==G.tutte_polynomial_by_contract_remove_edges(x,y) and G.tutte_polynomial_by_number_of_components(x,y)(2,2)==2**(G.Size()):
        a['T(2,2)'].append(G.tutte_polynomial_by_number_of_components(2,2))
a

{'Graphs': ['Empty graph (n=1)',
  'Empty graph (n=5)',
  'Empty graph (n=10)',
  'Path graph P2',
  'Path graph P5',
  'Path graph P10',
  'Cycle graph C10',
  'Pan graph C7+K1',
  'Sunlet graph (n=10)',
  'Star graph S7',
  'Bull graph',
  'Butterfly graph',
  'Cubical graph',
  'Diamond graph',
  'Ladder graph (n=5)',
  'Wheel graph (n=6)',
  'Gear graph (n=4)'],
 'T(1,1)': [1, 1, 1, 1, 1, 1, 10, 7, 5, 1, 3, 9, 384, 8, 209, 121, 192],
 'T(1,2)': [1, 1, 1, 1, 1, 1, 11, 8, 6, 1, 4, 16, 1083, 14, 479, 462, 431],
 'T(2,1)': [1,
  1,
  1,
  2,
  16,
  512,
  1023,
  254,
  992,
  128,
  28,
  49,
  2656,
  24,
  6240,
  462,
  3102],
 'T(2,2)': [1,
  1,
  1,
  2,
  16,
  512,
  1024,
  256,
  1024,
  128,
  32,
  64,
  4096,
  32,
  8192,
  1024,
  4096]}

In [22]:
ad=pd.DataFrame(a)
ad

,Graphs,Tutte Polynomial,"T(1,1)","T(1,2)","T(2,1)","T(2,2)"
0,Empty graph (n=1),"Poly(1, x, y, domain='ZZ')",1,1,1,1
1,Empty graph (n=5),"Poly(1, x, y, domain='ZZ')",1,1,1,1
2,Empty graph (n=10),"Poly(1, x, y, domain='ZZ')",1,1,1,1
3,Path graph P2,"Poly(x, x, y, domain='ZZ')",1,1,2,2
4,Path graph P5,"Poly(x**4, x, y, domain='ZZ')",1,1,16,16
5,Path graph P10,"Poly(x**9, x, y, domain='ZZ')",1,1,512,512
6,Cycle graph C10,Poly(x**9 + x**8 + x**7 + x**6 + x**5 + x**4 +...,10,11,1023,1024
7,Pan graph C7+K1,Poly(x**7 + x**6 + x**5 + x**4 + x**3 + x**2 +...,7,8,254,256
8,Sunlet graph (n=10),"Poly(x**9 + x**8 + x**7 + x**6 + x**5*y, x, y,...",5,6,992,1024
9,Star graph S7,"Poly(x**7, x, y, domain='ZZ')",1,1,128,128


In [23]:
ad.to_excel('data4.xlsx',sheet_name='Sheet1')